## testing hw

In [195]:
import numpy as np
import pandas as pd
from scipy import sparse
from cf import *

## Encoding rating data
Here are our very small subset of fake data to get us started.

In [196]:
# The first row says that user 1 reated movie 11 with a score of 4
!cat tiny_training.csv 

userId,movieId,rating
11,1,4
11,23,5
2,23,5
2,4,3
31,1,4
31,23,4
4,1,5
4,3,2
52,1,1
52,3,4
61,3,5
7,23,1
7,3,3


In [197]:
df = pd.read_csv("tiny_training.csv")
# df, num_users, num_movies = encode_data(df)

In [198]:
name2idx,df["userId"],num_users = proc_col(df["userId"])
df

,userId,movieId,rating
0,0,1,4
1,0,23,5
2,1,23,5
3,1,4,3
4,2,1,4
5,2,23,4
6,3,1,5
7,3,3,2
8,4,1,1
9,4,3,4


In [199]:
def encode_data(df):
    """Encodes rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      train_csv: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies
      
    """
    ### BEGIN SOLUTION
    name2idx,df["userId"],num_users = proc_col(df["userId"])
    name2idx_2,df["movieId"],num_movies = proc_col(df["movieId"])
    ### END SOLUTION
    return df, num_users, num_movies

In [200]:
df, num_users, num_movies = encode_data(df)
df

,userId,movieId,rating
0,0,0,4
1,0,1,5
2,1,1,5
3,1,2,3
4,2,0,4
5,2,1,4
6,3,0,5
7,3,3,2
8,4,0,1
9,4,3,4


In [201]:
assert(num_users == 7)


In [202]:
assert(num_movies == 4)

In [203]:
np.testing.assert_equal(df["userId"].values, np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6]))

In [204]:
np.testing.assert_equal(df["movieId"].values, np.array([0, 1, 1, 2, 0, 1, 0, 3, 0, 3, 3, 1, 3]))

## Initializing parameters

In [118]:
# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

array([[3.55790894, 4.69774849, 0.92361109, 1.58739544, 3.00593239],
       [4.69774849, 7.44656163, 1.18135616, 2.64524868, 4.74559066],
       [0.92361109, 1.18135616, 0.24548062, 0.34025121, 0.69616965],
       [1.58739544, 2.64524868, 0.34025121, 1.61561   , 2.41361975],
       [3.00593239, 4.74559066, 0.69616965, 2.41361975, 3.82505541],
       [2.02000808, 3.29656257, 0.43174569, 2.065911  , 3.07264619],
       [2.07691001, 3.02887291, 0.53270924, 1.02482544, 1.90251125]])

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [205]:
df = pd.read_csv("tiny_training.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

In [206]:
print(Y)

  (0, 0)	4
  (2, 0)	4
  (3, 0)	5
  (4, 0)	1
  (0, 1)	5
  (1, 1)	5
  (2, 1)	4
  (6, 1)	1
  (1, 2)	3
  (3, 3)	2
  (4, 3)	4
  (5, 3)	5
  (6, 3)	3


## Calculating the cost function

In [ ]:
def sparse_multiply(df, emb_user, emb_movie):
    """ This function returns U*V^T element wise multi by R as a sparse matrix.
    
    It avoids creating the dense matrix U*V^T
    """
    
    df["Prediction"] = np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)
    return df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="Prediction")

In [207]:
df["userId"].values

array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6])

In [211]:
emb_movie.shape

(7867, 3)

In [208]:
emb_movie[df["movieId"].values]

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [209]:
df["movieId"].values

array([0, 1, 1, 2, 0, 1, 0, 3, 0, 3, 3, 1, 3])

In [194]:
emb_movie.shape

(7867, 3)

In [186]:
np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)

array([3.40885995, 2.0655048 , 0.82296001, 0.69616965, 7.15706307,
       4.89139359, 4.74559066, 2.41361975, 4.03343148, 2.13164169,
       3.29218077, 2.26571732, 1.561454  ])

In [ ]:
df["Prediction"] = np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)

In [121]:
sparse_multiply(df, emb_user, emb_movie).data

array([3.55790894, 0.92361109, 1.58739544, 3.00593239, 4.69774849,
       7.44656163, 1.18135616, 3.02887291, 1.18135616, 1.61561   ,
       2.41361975, 2.065911  , 1.02482544])

In [122]:
def cost(df, emb_user, emb_movie):
    """ Computes mean square error

    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]

    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies

    Returns:
      error(float): this is the MSE
    """
    # BEGIN SOLUTION
    prediction = sparse_multiply(df, emb_user, emb_movie).data
    Y = df2matrix(df, nrows=emb_user.shape[0], ncols=emb_movie.shape[0]).data
    error = np.mean((Y - prediction)**2)
    return error

In [193]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 2.23)

## Calculating gradient

In [124]:
def gradient(df, Y, emb_user, emb_movie):
    """ Computes the gradient.
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    ### BEGIN SOLUTION
    N = df.shape[0]
    delta = Y.toarray() - sparse_multiply(df, emb_user, emb_movie).toarray()
    grad_user = (-2.00)/N*np.dot(delta,emb_movie)
    grad_movie = (-2.00)/N*np.dot(delta.transpose(),emb_user)
    ### END SOLUTION
    return grad_user, grad_movie

In [125]:
df2matrix(df, emb_user.shape[0], emb_movie.shape[0]).toarray()

array([[4, 5, 0, 0],
       [0, 5, 3, 0],
       [4, 4, 0, 0],
       [5, 0, 0, 2],
       [1, 0, 0, 4],
       [0, 0, 0, 5],
       [0, 1, 0, 3]])

In [174]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
grad_user, grad_movie = gradient(df, Y, emb_user, emb_movie)

In [127]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [128]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Gradient descent with momentum

In [131]:
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.
    0.9 means average last 10 gradients 
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    ### BEGIN SOLUTION
    emb_user_momen, emb_movie_momen = gradient(df, Y, emb_user, emb_movie)
    for i in range(iterations):
        emb_user_grad, emb_movie_grad = gradient(df, Y, emb_user, emb_movie)
        
        emb_user_momen = 0.9*emb_user_momen + (1-0.9)*emb_user_grad
        emb_user = emb_user- learning_rate*emb_user_momen
        
        emb_movie_momen = 0.9*emb_movie_momen + (1-0.9)*emb_movie_grad
        emb_movie = emb_movie- learning_rate*emb_movie_momen
        
        
        if (i+1)%50 == 0:
            print("training cost:",cost(df, emb_user, emb_movie))
            if df_val is None:
                print("None")
            else: 
                print("validation cost:", cost(df_val,emb_user, emb_movie))
    
    ### END SOLUTION
    return emb_user, emb_movie

In [146]:
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.

    Prints training cost and validation cost (if df_val is not None) every 50 iterations.

    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    # BEGIN SOLUTION
    moment_user = 0
    moment_movie = 0
    for i in range(iterations):
        grad_user, grad_movie = gradient(df, Y, emb_user, emb_movie)
        moment_user = 0.9*moment_user + (1-0.9)*grad_user
        moment_movie = 0.9*moment_movie + (1-0.9)*grad_movie
        emb_user = emb_user-learning_rate*moment_user
        emb_movie = emb_movie-learning_rate*moment_movie
        if (i+1) % 50 == 0:
            if df_val is None:
                cost_train = cost(df, emb_user, emb_movie)
                print(i+1,"training cost:", cost_train," None")
            else:
                cost_train = cost(df, emb_user, emb_movie)
                cost_val = cost(df_val, emb_user, emb_movie)
                print(i+1,"training cost:", cost_train,"validation cost:", cost_val)
    # END SOLUTION
    return emb_user, emb_movie

In [147]:
emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01)

50 training cost: 1.991816782983179  None
100 training cost: 1.0551363096429935  None
150 training cost: 0.7377729563134163  None
200 training cost: 0.5539263202812025  None


In [148]:
train_mse = cost(df, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.55)

## Predicting on new data
Now we should write a function that given new data is able to predict ratings. First we write a function that encodes new data. If a new user or item is present that row should be remove. Collaborative Filtering is not good at handling new users or new items. To help with this task, you could write a an auxiliary function similar to `proc_col`.

In [149]:
a=df['rating'].unique()
df[df["rating"].isin(a)]

,userId,movieId,rating,Prediction
0,0,0,4,4.586052
1,0,1,5,4.451704
2,1,1,5,5.276254
3,1,2,3,2.633234
4,2,0,4,3.921229
5,2,1,4,3.908405
6,3,0,5,3.739579
7,3,3,2,3.312920
8,4,0,1,1.883755
9,4,3,4,3.447729


In [150]:
def proc_col_val(col,df_val,df_train):
    """Encodes a pandas column with values between 0 and n-1.
     as the order of the first time a index appear
    where n = number of unique values
    """
    uniq = df_train[col].unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return np.array([name2idx[x] for x in df_val[col]])

In [151]:
def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    ### BEGIN SOLUTION
    #filter out new_items
    df_val = df_val[df_val["userId"].isin(df_train["userId"])]
    df_val = df_val[df_val["movieId"].isin(df_train["movieId"])]
    #encoding
    df_val["userId"] = proc_col_val("userId",df_val,df_train)
    df_val["movieId"] = proc_col_val("movieId",df_val,df_train)
    ### END SOLUTION
    return df_val

In [152]:
df_t = pd.read_csv("tiny_training.csv")
df_v = pd.read_csv("tiny_val.csv")
df_v = encode_new_data(df_v, df_t)

In [153]:
assert(len(df_v.userId.unique())==2)

In [154]:
assert(len(df_v) == 2)

## Putting it all together
For this part you should get data from here
`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

In [155]:
# !wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [156]:
# sorting by timestamp
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
data = data.sort_values(by=['timestamp'])

In [157]:
data.head()

,userId,movieId,rating,timestamp
66719,429,595,5.0,828124615
66716,429,588,5.0,828124615
66717,429,590,5.0,828124615
66718,429,592,5.0,828124615
66712,429,432,3.0,828124615


In [158]:
m = int(data.shape[0]*0.8)
data.shape, m

((100836, 4), 80668)

In [78]:
m = int(data.shape[0]*0.8)
data.shape, m

((100836, 4), 80668)

In [159]:
def proc_col(col):
    """Encodes a pandas column with values between 0 and n-1.
     as the order of the first time a index appear
    where n = number of unique values
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [160]:
train = data[:m].copy()
val = data[m:].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))

20168 1311


In [161]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000,
                                       learning_rate=1, df_val=df_val)

50 training cost: 9.678300503156466 validation cost: 10.397601309202344
100 training cost: 6.537466192555465 validation cost: 8.386523484535143
150 training cost: 4.546684280431599 validation cost: 6.990346795261098
200 training cost: 3.5033212064585033 validation cost: 6.148866866928117
250 training cost: 2.856515050130016 validation cost: 5.552528874692432
300 training cost: 2.418518126827779 validation cost: 5.072475084198574
350 training cost: 2.1063425159680187 validation cost: 4.667197107878037
400 training cost: 1.8748808366877387 validation cost: 4.315775355366121
450 training cost: 1.6976210533617162 validation cost: 4.006112971431078
500 training cost: 1.5581460435508805 validation cost: 3.7305356522521653
550 training cost: 1.4458713206767109 validation cost: 3.483701158123081
600 training cost: 1.353739041286662 validation cost: 3.261605315341615
650 training cost: 1.2769004981724286 validation cost: 3.0610863831184467
700 training cost: 1.2119324116196601 validation cost: 

In [162]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

0.7113513270477632 1.140599450059145


In [163]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.71)

In [164]:
val_mse = cost(df_val, emb_user, emb_movie)
assert(np.around(val_mse, decimals=2) == 1.14)

In [165]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=200, learning_rate=1, df_val=df_val)

50 training cost: 9.678300503156466 validation cost: 10.397601309202344
100 training cost: 6.537466192555465 validation cost: 8.386523484535143
150 training cost: 4.546684280431599 validation cost: 6.990346795261098
200 training cost: 3.5033212064585033 validation cost: 6.148866866928117


In [166]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

3.5033212064585033 6.148866866928117


In [167]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=1) == 3.5)

In [168]:
val_mse = cost(df_val, emb_user, emb_movie)
assert(np.around(val_mse, decimals=1) == 6.1)